In [1]:
import os
import lief

import pymysql
from warnings import filterwarnings

_connection = None

def get_connection(db_config):
    """
    get db connection
    :return:
    """
    global _connection
    if _connection is None:
        _connection = pymysql.connect(host=db_config['host'], user=db_config['username'],
                                      password=db_config['password'],
                                      db=db_config['db'], charset="utf8")
        filterwarnings('ignore', category=pymysql.Warning)

    return _connection


def close():
    """
    close DB connection
    :return:
    """
    global _connection
    if _connection is not None:
        _connection.close()
    _connection = None

In [2]:
db = {
    'host': '172.26.187.242',
    'username': 'malware_r',
    'password': 'GEg22v2O7jbfWhb3',
    'db': 'malware'
}

In [3]:
def common_get(table, condition, columns={"*"}, limit=None):
    """
    common get data function
    :param table:
    :param condition:
    :param columns:
    :param limit:
    :return:
    """
    global _connection
    if _connection is None:
        raise Exception("please init db connect first")

    cursor = _connection.cursor()
    cursor.execute("SET NAMES utf8mb4")

    ret = []

    sql = "SELECT %s FROM {table} WHERE 1" % ", ".join(columns)
    for item in condition:
        sql += " AND " + item

    sql = sql.format(table=table)
    if limit is not None:
        sql += " LIMIT " + str(limit)
    cursor.execute(sql)

    field_names = [i[0] for i in cursor.description]

    for row in cursor:
        temp = {}
        for key in range(len(row)):
            temp[field_names[key]] = row[key]
        ret.append(temp)

    cursor.close()
    # _connection.close()

    return ret

In [7]:
get_connection(db)
table_suffix = ["0","1","2","3","4","5","6","7","8","9","A","B","C","D","E","F"]
res = []
for suffix in table_suffix:
    res.extend(common_get("mw_index_2017_" + suffix, ["mw_file_size > 1024"], 
                          {"mw_file_hash", "mw_file_size", "mw_file_directory"}))
close()

In [8]:
len(res)

1423004

In [9]:
# import lief

base_path = "/hdd1/raw_pe_data/2017/{0}/{1}_{2}"
idx = 0
for item in res:
    file_path = base_path.format(item["mw_file_directory"], item["mw_file_hash"], item["mw_file_size"])
    lief_binary = lief.PE.parse(file_path)
    print(file_path)
    for section in lief_binary.sections:
        print(section)
    idx += 1
    if idx > 2:
        break

/hdd1/raw_pe_data/2017/201701/004A6FEA496A60BF1D12E613C100C91E582517D32B9691F7358B48A00A8736DB_7168
.text     eef       1000      1000      400       0         6.34585   CNT_CODE - MEM_EXECUTE - MEM_READ
.data     ac        2000      200       1400      0         2.82428   CNT_INITIALIZED_DATA - MEM_READ - MEM_WRITE
.rsrc     3e8       3000      400       1600      0         3.31465   CNT_INITIALIZED_DATA - MEM_READ
.reloc    14c       4000      200       1a00      0         5.53136   CNT_INITIALIZED_DATA - MEM_DISCARDABLE - MEM_READ
/hdd1/raw_pe_data/2017/201701/055ABABEAA7592D6B2F93502B1986130644542E084AA5B1992CB876155A84EFB_32256
.text     774       2000      800       200       0         4.87809   CNT_CODE - MEM_EXECUTE - MEM_READ
.rsrc     7148      4000      7200      a00       0         5.4705    CNT_INITIALIZED_DATA - MEM_READ
.reloc    c         c000      200       7c00      0         1.58496   CNT_INITIALIZED_DATA - MEM_DISCARDABLE - MEM_READ
/hdd1/raw_pe_data/2017/201701/065